<a href="https://colab.research.google.com/github/bae3559/2021summerintern/blob/main/ResNet_withTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##ResNet 

```
torchvision.models.resnet
```

- resnet(18,34,50,101,152)를 다 만들 수 있도록 되어있음.
- 3x224x224 입력을 기준으로 만들도록 되어있음.
- input size가 다른 경우 ResNet를 적용하려면 어떻게 할 수 있을지 생각해보기. 


In [1]:
## ResNet  


import torch.nn as nn
import torch.utils.model_zoo as model_zoo


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

In [ ]:
class BasicBlock(nn.Module):
   def __init__(self,inplanes, planes, stride=1, downsample=None):
     super(BasicBlock, self).__init__()

     self.conv1 = conv3x3(inplanes, planes, stride)  # stride 값을 받은 걸 씀 
     self.bn1 = nn.BatchNorm2d(planes)
     self.relu = nn.ReLU(inplace=True)
     self.conv2 = conv3x3(planes,planes) # stride 무조건 1
     self.bn2 = nn.BatchNorm2d(planes)
     self.downsample = downsample
     self.stride = stride

    def forward(self, x):

      # if x.shape = 3x64x64
      identity = x   #identity.shape = 3x64x64
      # if stride=2  
      out = self.conv1(x) # padding :1 , out.shape = 3x32x32
      out = self.bn1(out)  # out.shape = 3x32x32
      out = self.relu(out)  # out.shape = 3x32x32

      out = self.conv2(out) # out.shape = 3x32x32
      out = self.bn2(out)  # out.shape = 3x32x32

      # 이렇게 되면 지금 identity 랑 out의 shape이 달라서 더 하는게 불가능해진다. 
      # 그럴 때, downsample을 해주면 됨! 

      if downsample != None:
        identity = self.downsample(x)

      out += identity 
      out = self.relu(out)

      return out

In [2]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
      super(Bottleneck, self).__init__()
      
      self.conv1 = conv1x1(inplanes, planes)
      self.bn1 = nn.BatchNorm2d(planes)
      self.conv2 = conv3x3(planes, planes,stride)
      self.bn2 = nn.BatchNorm2d(planes)
      self.conv3 = conv1x1(planes, planes * self.expansion)
      self.bn3 = nn.BatchNorm2d(planes * self.expansion)

      self.downsample = downsample
      self.relu = nn.ReLU(inplace=True)
      self.stride = stride

    def forward(self, x):

      identity = x

      out = self.conv1(x)
      out = self.bn1(out)
      out = self.relu(out)

      out = self.conv2(out)
      out = self.bn2(out)
      out = self.relu(out)

      out = self.conv3(out)
      out = self.bn3(out)
      
      if downsample != None:
        identity = self.downsample(x)

      out += identity 
      out = self.relu(out)

      return out

In [3]:
def resenet18(pretrained=False, **kwargs):

  """
    Args:
        pretrained(bool): If True, returns a model pre-trained on ImageNet
  """
  model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
  if pretrained:
    model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
  return model

def resnet50(pretrained=False, **kwargs):
  model = ResNet(Bottleneck,[3, 4, 6, 3], **kwargs)
  if pretrained:
    model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
  return model

In [ ]:
class ResNet(nn.Module):
  #model = ResNet(Bottleneck,[3, 4, 6, 3], **kwargs) resnet50
  def __init__(self, block, layers, num_classes=1000, zero_init_residual=False):
    super(ResNet, self).__init__()
    self.inplanes = 64
    
    #inputs = 3x224x224
    self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False) 
    # outputs = self.conv1(inputs)라면, outputs.shape = [64x112x112]
    self.bn1 = nn.BatchNorm2d(64)
    self.relu = nn.ReLU(inplace=True)
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    #  64x56x56 
 
    self.layer1 = self._make_layer(block, 64, layers[0]'''3''')
    self.layer2 = self._make_layer(block, 128, layers[1]'''4''', stride=2)
    self.layer3 = self._make_layer(block, 256, layers[2]'''6''', stride=2)
    self.layer4 = self._make_layer(block, 512, layers[3]'''3''', stride=2)
    
    self.avgpool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(512 * block.expansion, num_classes)

    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
      elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight,1)
        nn.init.constant_(m.bias,1)

    if zero_init_residual:
      for m in self.modules():  
        if isinstance(m, Bottleneck):
          nn.init.constant_(m.bn2.weight,0)
        elif isinstance(m, BasicBlock):
          nn.init.constant_(m.bn2.weight,0)

  def _make_layer(self, block, planes, blocks, stride=1):
    downsample = None
    if stride != 1 or self.inplanes != planes * block.exmpansion:
      downsample = nn.Sequential(
          conv1x1(self.inplanes, planes*block.expansion, stride), 
          nn.BatchNorm2d(planes*block.expansion) )
      layers=[]


0
